In [1]:
import random
import tensorflow as tf
import numpy as np

In [2]:
upper_bound=10
lower_bound=1
symbol = {
    '+':1000,
    '-':2000,
    '*':3000,
    '/':4000,
}

symbol_s = {
    1000:'+',
    2000:'-',
    3000:'*',
    4000:'/',
}

In [3]:
def convertToArray(s):
    a = []
    prev_index=0
    for i in range(len(s)):
        if s[i] in symbol.keys():
            a.append(int(s[prev_index:i]))
            a.append(symbol[s[i]])
            prev_index = i+1
            
    a.append(int(s[prev_index:len(s)]))
    return a
    
#convert equation to array

def convertToString(a):
    s=""
    for i in a:
        if i in symbol_s.keys():
            s=s+(symbol_s[i])
        else:
            s=s+(str(i))
    return s

In [4]:
ss = convertToArray("11+2/344*4")
print(ss)
print(convertToString(ss))

[11, 1000, 2, 4000, 344, 3000, 4]
11+2/344*4


In [5]:
def generateEquation():
    eq = []
    eq.append(random.randint(lower_bound, upper_bound))
    for i in range(4):
        eq.append(random.randint(1, 4)*1000)
        eq.append(random.randint(lower_bound, upper_bound))
    return eq

In [6]:
test = generateEquation()
print(convertToString(test))

5*7+6*3*2


In [7]:
def generateStep(a):  
    mark=[[],[]] # step1 and step 2 marks
    sol = [a]
    
    n = len(a)//2
    
    prev = a
    for i in range(n-1,-1,-1):
        x = random.randint(0,i)
        #print("i="+str(i)+"x="+str(x))
        #print(prev[:2*x],prev[2*x+3:])
        
        step = prev[:2*x]
        
        #process step
        sym = a[2*x+1]
        #print(sym)
        error = random.randint(0,4)//4
        if sym == 1000:
            step.append(prev[2*x] + prev[2*x+2]+error)
            mark[0].append(0) if (3000 in prev) or (4000 in prev) else mark[0].append(1)
            mark[1].append(0) if error != 0 else mark[1].append(1)
        elif sym == 2000:
            step.append(prev[2*x] - prev[2*x+2]+error)
            mark[0].append(0) if (3000 in prev) or (4000 in prev) else mark[0].append(1)
            mark[1].append(0) if error != 0 else mark[1].append(1)
        elif sym == 3000:
            step.append(prev[2*x] * prev[2*x+2]+error)
            mark[0].append(1);
            mark[1].append(0) if error != 0 else mark[1].append(1)
        elif sym == 4000:
            step.append(prev[2*x] / prev[2*x+2]+error)
            mark[0].append(1);
            mark[1].append(0) if error != 0 else mark[1].append(1)
        
        step += prev[2*x+3:]
        step += [0]*(len(a)-len(step))
        #print(step)
        sol.append(step)
        prev = step
    
    
    
    
    
    return (sol,mark)

In [8]:
test = generateEquation()
for step in generateStep(test):
    for i in step:
        print(i)

[1, 2000, 5, 2000, 4, 4000, 1, 4000, 6]
[1, 2000, 2, 4000, 1, 4000, 6, 0, 0]
[1, 2000, 2, 4000, 6, 0, 0, 0, 0]
[-1, 4000, 6, 0, 0, 0, 0, 0, 0]
[-6, 0, 0, 0, 0, 0, 0, 0, 0]
[0, 0, 0, 0]
[0, 0, 1, 0]


In [27]:
size = 5000

traindata=[]
trainlabel1=[]
trainlabel2=[]

for i in range(size):
    try:
        (sol, mark) = generateStep(generateEquation())
        for j in range(4):
            traindata.append(sol[j:j+2])
        trainlabel1+=mark[0]
        trainlabel2+=mark[1]
    except:
        i=i-1

In [61]:
model1 = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(72, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(16, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(2, activation=tf.nn.softmax)])

model1.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [62]:
model1.fit(traindata, trainlabel1, epochs=10)

Epoch 1/10
620/620 [==============================] - 0s 496us/step - loss: 8.8899 - accuracy: 0.6925
Epoch 2/10
620/620 [==============================] - 0s 489us/step - loss: 1.4000 - accuracy: 0.7258
Epoch 3/10
620/620 [==============================] - 0s 482us/step - loss: 0.8983 - accuracy: 0.7452
Epoch 4/10
620/620 [==============================] - 0s 490us/step - loss: 0.7224 - accuracy: 0.7629
Epoch 5/10
620/620 [==============================] - 0s 479us/step - loss: 0.6776 - accuracy: 0.7714
Epoch 6/10
620/620 [==============================] - 0s 479us/step - loss: 0.6289 - accuracy: 0.7823
Epoch 7/10
620/620 [==============================] - 0s 490us/step - loss: 0.6042 - accuracy: 0.7800
Epoch 8/10
620/620 [==============================] - 0s 480us/step - loss: 0.4700 - accuracy: 0.7926
Epoch 9/10
620/620 [==============================] - 0s 480us/step - loss: 0.4397 - accuracy: 0.7993
Epoch 10/10
620/620 [==============================] - 0s 491us/step - loss: 0.433

In [63]:
model2 = tf.keras.models.Sequential([tf.keras.layers.Flatten(), 
                                    tf.keras.layers.Dense(72, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(16, activation=tf.nn.relu), 
                                    tf.keras.layers.Dense(2, activation=tf.nn.softmax)])

model2.compile(optimizer = tf.keras.optimizers.Adam(),
              loss = 'sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [64]:
model2.fit(traindata, trainlabel2, epochs=10)

Epoch 1/10
620/620 [==============================] - 0s 485us/step - loss: 25.0499 - accuracy: 0.6754
Epoch 2/10
620/620 [==============================] - 0s 479us/step - loss: 10.3256 - accuracy: 0.6777
Epoch 3/10
620/620 [==============================] - 0s 479us/step - loss: 8.3671 - accuracy: 0.6761
Epoch 4/10
620/620 [==============================] - 0s 485us/step - loss: 6.8817 - accuracy: 0.6820
Epoch 5/10
620/620 [==============================] - 0s 480us/step - loss: 6.7197 - accuracy: 0.6831
Epoch 6/10
620/620 [==============================] - 0s 490us/step - loss: 5.4650 - accuracy: 0.6839
Epoch 7/10
620/620 [==============================] - 0s 495us/step - loss: 5.4880 - accuracy: 0.6841
Epoch 8/10
620/620 [==============================] - 0s 487us/step - loss: 4.3785 - accuracy: 0.6811
Epoch 9/10
620/620 [==============================] - 0s 489us/step - loss: 3.9528 - accuracy: 0.6838
Epoch 10/10
620/620 [==============================] - 0s 488us/step - loss: 3.9

In [65]:
test_size = 100

testdata=[]
testlabel1=[]
testlabel2=[]

for i in range(size):
    try:
        (sol, mark) = generateStep(generateEquation())
        for j in range(4):
            testdata.append(sol[j:j+2])
        testlabel1+=mark[0]
        testlabel2+=mark[1]
    except:
        i=i-1

In [66]:
model1.evaluate(testdata, testlabel1)

620/620 [==============================] - 0s 384us/step - loss: 0.4377 - accuracy: 0.8011


[0.43770289421081543, 0.8010687828063965]

In [67]:
model2.evaluate(testdata, testlabel2)

620/620 [==============================] - 0s 411us/step - loss: 7.7883 - accuracy: 0.6346


[7.788341522216797, 0.6345533132553101]